In [1]:
from sktime.transformers.compose import ColumnConcatenator 
from sktime.classifiers.compose import TimeSeriesForestClassifier
from sktime.classifiers.dictionary_based.boss import BOSSEnsemble
from sktime.classifiers.compose import ColumnEnsembleClassifier
from sktime.classifiers.shapelet_based import ShapeletTransformClassifier
from sktime.datasets import load_basic_motions
from sktime.pipeline import Pipeline

import numpy as np
import pandas as pd

In [18]:
df = pd.read_csv("pre_processed_data.csv") 
df.rename(columns={'Bleeding': 'y'}, inplace=True)
df.head(n=10)

,wind_max,wind_ave,rainfall_height,rainfall_form,sunny_hours,snow_height,bedeckungsgrades,dampfdruckes,luftdruck,temp,humidity,air_temp_max,air_temp_min,ground_temp_min,y
0,14.2,4.6,15.0,4.0,0.2,0.0,8.0,7.1,983.6,3.6,86.0,10.6,-0.8,-0.8,0
1,21.6,5.5,9.0,4.0,0.2,0.0,7.8,10.2,967.9,9.2,88.0,11.8,6.8,6.0,0
2,18.7,4.4,7.4,4.0,0.0,0.0,7.8,8.5,976.2,6.2,90.0,8.5,1.2,1.0,0
3,11.0,5.3,0.0,4.0,1.6,0.0,6.8,4.4,988.0,-1.3,79.0,1.4,-2.7,-2.7,0
4,7.8,2.7,1.2,4.0,0.6,0.0,7.8,4.1,993.2,-3.2,84.0,-1.7,-4.6,-4.8,1
5,9.7,3.7,0.1,4.0,0.3,2.0,6.3,4.4,994.0,-2.6,86.0,-0.9,-5.0,-6.6,0
6,7.8,3.1,0.0,4.0,2.5,2.0,7.0,3.2,1000.0,-5.4,78.0,-4.2,-6.3,-6.9,1
7,7.8,3.1,0.0,4.0,2.5,2.0,7.0,3.2,1000.0,-5.4,78.0,-4.2,-6.3,-6.9,1
8,8.5,3.0,0.0,0.0,7.5,2.0,3.3,2.3,997.6,-8.0,70.0,-5.3,-10.7,-12.5,0
9,9.2,3.5,0.0,0.0,7.2,2.0,0.5,2.0,999.4,-8.7,62.0,-5.2,-11.1,-11.8,1


In [10]:
#testing dataset (both 0 and 1)
df_train = df[:4925]
x_0_1 = df_train.loc[:, df.columns != 'y']
y_0_1 = df_train.loc[:, df.columns == 'y']

In [23]:
# multi-class target variable 
np.unique(y_0_1)

array([0, 1], dtype=int64)

In [24]:
steps = [
    ('concatenate', ColumnConcatenator()), 
    ('classify', TimeSeriesForestClassifier(n_estimators=100))]
clf = Pipeline(steps)
clf.fit(x_0_1, y_0_1)

IndexError: tuple index out of range

In [30]:
clf = ColumnEnsembleClassifier(estimators=[
    ("TSF0", TimeSeriesForestClassifier(n_estimators=100), [0]),
    ("BOSSEnsemble3", BOSSEnsemble(ensemble_size=3), [3]),
])
clf.fit(x_0_1, y_0_1)

IndexError: tuple index out of range

In [36]:
clf = ShapeletTransformClassifier(time_contract_in_mins=0.5)
clf.fit(x_0_1, np.array(y_0_1))

TypeError: object of type 'numpy.float64' has no len()